In [ ]:
import soccerdata as sd
import pandas as pd
import numpy as np
from datetime import datetime
import os
from pathlib import Path

# --- CONFIGURAÇÕES GERAIS ---
SEASON = "2526"  # Temporada 2025/2026
LEAGUE = "ENG-Premier League"
# Define a data "de corte". Como você está simulando o projeto hoje:
CURRENT_DATE = datetime.now() 

# Configura pandas para mostrar mais colunas no terminal
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

def extract_data():
    """
    Baixa dados brutos do FBref via soccerdata.
    Retorna dois DataFrames: Stats Básicos e Stats Avançados.
    """
    print(f"🔄 [1/4] Iniciando conexão com FBref para a temporada {SEASON}...")

    cache_path = Path("./data/cache")
    
    # Inicializa o scraper
    # data_dir define onde o soccerdata guarda o cache (evita baixar tudo de novo)
    fbref = sd.FBref(leagues=LEAGUE, seasons=SEASON, data_dir=cache_path)
    
    print("⬇️  [2/4] Baixando estatísticas de Goleiros (Shot Stopping)...")
    keepers = fbref.read_player_match_stats(stat_type="keepers")
    
    print("⬇️  [3/4] Baixando estatísticas Avançadas (xG, Cruzamentos, Sweeper)...")
    keepers_adv = fbref.read_player_match_stats(stat_type="keepers_adv")
    
    return keepers, keepers_adv

def process_and_merge(df_basic, df_adv):
    """
    Limpa, filtra datas futuras, remove reservas e une os datasets.
    """
    print("⚙️  [4/4] Processando e unificando os dados...")
    
    # 1. Resetar Index (O soccerdata devolve MultiIndex complexo)
    basic = df_basic.reset_index()
    adv = df_adv.reset_index()
    
    # 2. Padronizar Datas
    basic['date'] = pd.to_datetime(basic['date'])
    adv['date'] = pd.to_datetime(adv['date'])
    
    # 3. FILTRO CRÍTICO: Remover jogos futuros e Goleiros que não jogaram
    # Isso limpa o "lixo" de rodadas que ainda não aconteceram na temporada 25/26
    basic = basic[(basic['date'] <= CURRENT_DATE) & (basic['minutes'] > 0)]
    adv = adv[(adv['date'] <= CURRENT_DATE) & (adv['minutes'] > 0)]
    
    # 4. Merge (União) dos DataFrames
    # Chaves para garantir que estamos unindo o mesmo jogador no mesmo jogo
    merge_keys = ['league', 'season', 'date', 'team', 'player', 'opponent']
    
    # Removemos colunas duplicadas do 'adv' antes do merge para evitar conflitos (ex: minutes, age)
    # Mantemos apenas as colunas chaves + as métricas exclusivas do advanced
    cols_to_use_adv = merge_keys + [col for col in adv.columns if col not in basic.columns and col not in merge_keys]
    
    df_full = pd.merge(
        basic,
        adv[cols_to_use_adv],
        on=merge_keys,
        how='left' # Mantém a base do 'basic'
    )
    
    return df_full

def feature_engineering_initial(df):
    """
    Renomeia colunas e cria as métricas base (PSxG +/-, %).
    """
    # Mapa de Renomeação (Do nome técnico do FBref para algo legível)
    cols_map = {
        'date': 'Date',
        'player': 'Player',
        'team': 'Team',
        'opponent': 'Opponent',
        'minutes': 'Minutes',
        # Métricas de Shot Stopping
        'performance_ga': 'GA',                 # Gols Sofridos
        'performance_saves': 'Saves',           # Defesas
        'expected_psxg': 'PSxG',                # Expected Goals Pós-Chute
        # Métricas Aéreas
        'crosses_opp_stp': 'Crosses_Stopped',   # Cruzamentos Interceptados
        'crosses_opp_opp': 'Crosses_Faced',     # Total Cruzamentos na área
        # Métricas de Saída (Sweeper)
        'sweeper_opa': 'OPA',                   # Ações Defensivas Fora da Área
        'passes_launch_completed': 'Long_Passes_Completed',
        'passes_launch_attempted': 'Long_Passes_Attempted'
    }
    
    # Filtra apenas colunas que existem
    valid_cols = [c for c in cols_map.keys() if c in df.columns]
    df_clean = df[valid_cols].copy()
    df_clean.rename(columns=cols_map, inplace=True)
    
    # --- CÁLCULOS MATEMÁTICOS BÁSICOS ---
    
    # 1. PSxG +/- (A métrica mais importante)
    # Quanto mais positivo, mais o goleiro "salvou" o time.
    df_clean['PSxG_Net'] = df_clean['PSxG'] - df_clean['GA']
    
    # 2. % de Cruzamentos (Tratando divisão por zero)
    df_clean['Cross_Stop_Pct'] = np.where(
        df_clean['Crosses_Faced'] > 0,
        (df_clean['Crosses_Stopped'] / df_clean['Crosses_Faced']) * 100,
        0.0
    )
    
    # 3. % de Lançamentos Longos
    df_clean['Long_Pass_Pct'] = np.where(
        df_clean['Long_Passes_Attempted'] > 0,
        (df_clean['Long_Passes_Completed'] / df_clean['Long_Passes_Attempted']) * 100,
        0.0
    )

    return df_clean

# --- BLOCO DE EXECUÇÃO ---
if __name__ == "__main__":
    # Garante que as pastas existem
    os.makedirs("data/processed", exist_ok=True)
    os.makedirs("data/cache", exist_ok=True)
    
    try:
        # 1. Extrair
        raw_basic, raw_adv = extract_data()
        
        # 2. Processar e Unir
        df_merged = process_and_merge(raw_basic, raw_adv)
        
        # 3. Limpar e Calcular Métricas Base
        df_final = feature_engineering_initial(df_merged)
        
        # 4. Salvar
        output_path = "data/processed/pl_goalkeepers_2526_matches.csv"
        df_final.to_csv(output_path, index=False)
        
        print(f"\n✅ Sucesso! Arquivo salvo em: {output_path}")
        print(f"📊 Total de atuações extraídas: {len(df_final)}")
        print("\n--- Preview das últimas atuações (Top 5) ---")
        print(df_final[['Date', 'Player', 'Team', 'Opponent', 'PSxG', 'GA', 'PSxG_Net']].tail())
        
    except Exception as e:
        print(f"\n❌ Erro durante a execução: {e}")

🔄 [1/4] Iniciando conexão com FBref para a temporada 2526...


[01/10/26 19:22:50] INFO     Saving cached data to ./data/cache                                      ]8;id=305121;file://c:\Users\jvamg\Documents\fds\powerranking\.venv\Lib\site-packages\soccerdata\_common.py\_common.py]8;;\:]8;id=36990;file://c:\Users\jvamg\Documents\fds\powerranking\.venv\Lib\site-packages\soccerdata\_common.py#263\263]8;;\


❌ Erro durante a execução: 'str' object has no attribute 'mkdir'
